### **M/M/1 Queue Simulation**

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jan 20 12:13:00 2022

@author: Yang Boyu

Part 1 code for Fast Simulation Project
"""

'\nCreated on Thu Jan 20 12:13:00 2022\n\n@author: Yang Boyu\n\nPart 1 code for Fast Simulation Project\n'

In [2]:
# package and settings
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
pd.set_option('expand_frame_repr', False)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180)

#### **1. Lindley's Recursion**

Target: 

Construct normal confidence interval for $E[W_n]$ with n=1, 10, 100, 1000, 10000 with interval width $\leq$ 0.05, where $\lambda = 0.5$ and $ \mu = 2$.

Compare the simulation result with the theoretical steady-state mean waiting time $\frac{\rho}{\mu - \lambda}$, where $\rho = \frac{\lambda}{\mu}$ is the traffic load or utilization rate for the server.

We simulate the M/M/1 queue based on the Lindley Recursion formula: $W_{n+1}=[W_n+V_n-T_n]^{+}$, where in M/M/1 queue, service time $V_n \sim exp(\mu)$ and interarrival time $T_n \sim exp(\lambda)$. 

In [9]:
# estimate expected waiting time using Crude Monte Carlo
# return expected simulated waiting time and 95% confidence interval
def wat_time_simulation(lam, mu, n) -> list:
    estimate = []
    if n == 1:
        return [0, 0, 0]
    while True:
        W = 0
        for _ in range(n-1):
            V = np.random.exponential(scale=1/mu, size=1)
            T = np.random.exponential(scale=1/lam, size=1)
            W = max(0, W+V-T)
        estimate.append(W)

        if ((len(estimate) > 50000)): break

    estimate_wat = np.mean(estimate)
    return [estimate_wat-1.96*np.std(estimate)/np.sqrt(len(estimate)), estimate_wat, estimate_wat+1.96*np.std(estimate)/np.sqrt(len(estimate))]

In [10]:
# simulate the result under lambda=1.5 and mu=0.2
np.random.seed(43)
wat = wat_time_simulation(lam=1.5, mu=2, n=1)
print("n=1: Estimated mean waiting time: %f; 95%% Confidence interval: [%f, %f]"%(wat[1], wat[0], wat[2]))

n=1: Estimated mean waiting time: 0.000000; 95% Confidence interval: [0.000000, 0.000000]


In [11]:
np.random.seed(43)
wat = wat_time_simulation(lam=1.5, mu=2, n=10)
print("n=10: Estimated mean waiting time: %f; 95%% Confidence interval: [%f, %f]"%(wat[1], wat[0], wat[2]))

n=10: Estimated mean waiting time: 0.854324; 95% Confidence interval: [0.844877, 0.863772]


In [12]:
np.random.seed(43)
wat = wat_time_simulation(lam=1.5, mu=2, n=100)
print("n=100: Estimated mean waiting time: %f; 95%% Confidence interval: [%f, %f]"%(wat[1], wat[0], wat[2]))

n=100: Estimated mean waiting time: 1.483175; 95% Confidence interval: [1.466490, 1.499860]


In [13]:
np.random.seed(43)
wat = wat_time_simulation(lam=1.5, mu=2, n=1000)
print("n=1000: Estimated mean waiting time: %f; 95%% Confidence interval: [%f, %f]"%(wat[1], wat[0], wat[2]))

n=1000: Estimated mean waiting time: 1.504443; 95% Confidence interval: [1.487524, 1.521363]


In [14]:
np.random.seed(43)
wat = wat_time_simulation(lam=1.5, mu=2, n=10000)
print("n=10000: Estimated mean waiting time: %f; 95%% Confidence interval: [%f, %f]"%(wat[1], wat[0], wat[2]))

n=10000: Estimated mean waiting time: 1.509648; 95% Confidence interval: [1.492566, 1.526730]


#### **2. Regenerative Method**

Use the regenerative cycle to estimate the steady-state expected waiting time, using the same parameters with interval width ≤ 0.05, i.e.,
$\lambda = 1.5$ and $\mu = 2$.

In [48]:
# generate i.i.d cycles to estimate expected waiting time
# output the estimated waiting time and 95% confidence interval
def regen_method(lam, mu, n) -> list:
    num_sigma = []
    num_waiting = []
    
    for _ in range(n):
        
        cycle_time = 1
        wat_time = 0
        W = 0
        while True:
            V = np.random.exponential(scale=1/mu, size=1)
            T = np.random.exponential(scale=1/lam, size=1)
            W = max(0, W+V-T)
            if W == 0:
                break
            else:
                cycle_time += 1
                wat_time += W[0]
                
        num_sigma.append(cycle_time)
        num_waiting.append(wat_time)
    estimate_wat = np.mean(num_waiting)/np.mean(num_sigma)

    S = np.sqrt(np.cov(num_waiting, num_sigma)[0][0]-2*estimate_wat*np.cov(num_waiting, num_sigma)[0][1]+np.cov(num_waiting, num_sigma)[1][1]*(estimate_wat**2))

    return [estimate_wat-1.96*S/np.mean(num_sigma)/np.sqrt(n), estimate_wat, estimate_wat+1.96*S/np.mean(num_sigma)/np.sqrt(n)]

In [49]:
np.random.seed(43)
regen = regen_method(1.5, 2, 1000000)
print("Regenerative method with number of cycles 1000000: Estimated mean waiting time: %f; 95%% Confidence interval: [%f, %f]"%(regen[1], regen[0], regen[2]))

Regenerative method with number of cycles 1000000: Estimated mean waiting time: 1.498611; 95% Confidence interval: [1.485029, 1.512194]


#### **3. Change of Measure/Importance Sampling**

Target:

- Consider two system settings, $\lambda = 0.9,\mu=2.7$ (light traffic) and $\lambda = 1.6,\mu=2$ (heavy traffic)
- Use the optimal change of measure $\lambda^{'} = \mu$ and $\mu^{'} = \lambda$ and another two change of measures about $\theta$ such that $\frac{\lambda}{\lambda + \theta}\times \frac{\mu}{\mu - \theta}=0.9$ or $1.1$ (take the solution near the optimal $\theta$)
- Try threshold $\gamma$ such that $P(W_{\infty} > \gamma) = \rho \times e^{(\mu - \lambda)\gamma}$ will be $0.001, 10^{-5}, 10^{-10}$

We first try regenerative method **without** using change of measure, that is $P(W_{\infty}>\gamma)=\frac{E[\sum_{n=1}^\sigma I_{W_n \geq \gamma}]}{E[\sigma]}$, and the try the regenerative method **with** possible change of measure. we try two cases (light traffic and heavy traffic), Theoretically, the tail probability $P(W_{\infty} > \gamma)$ will be $\rho e^{-(\mu-\lambda)\gamma}$, which is 0.05599 ($\lambda =0.9,\mu =2.7,\gamma = 1$), 0.0091079($\lambda =0.9,\mu =2.7,\gamma = 2$), 0.161517($\lambda =1.6,\mu =2,\gamma = 4$), and 0.10826822($\lambda =1.6,\mu =2,\gamma = 5$). We will try different parameters and see if our estimation gets closer to the true value.

In [31]:
# estimate the tail probability without change of measure
# return the estimated numerator, denominator, and the tail probability
def importance_sampling_without_change(lam, mu, gamma, n) -> list:
    num_sigma = []
    num_waiting = []

    for i in range(n):
        
        cycle_time = 1
        exceed_indicator = 0
        W = 0
        while True:
            V = np.random.exponential(scale=1/mu, size=1)
            T = np.random.exponential(scale=1/lam, size=1)
            W = max(0, W+V-T)
            if W == 0:
                break
            else:
                cycle_time += 1
                if W >= gamma:
                    exceed_indicator += 1    
                
        num_sigma.append(cycle_time)
        num_waiting.append(exceed_indicator)
    return [np.mean(num_waiting), np.mean(num_sigma), np.mean(num_waiting)/np.mean(num_sigma)]

In [42]:
# estimate the tail probability with change of measure

# this function estimate the denominator, i.e. the cycle length without change of measure
# return the list of the estimate
def deno_estimate(lam, mu, n) -> list:
    num_sigma = []
    for i in range(n):
        cycle_time = 1
        W = 0
        while True:
            V = np.random.exponential(scale=1/mu, size=1)
            T = np.random.exponential(scale=1/lam, size=1)
            W = max(0, W+V-T)
            if W == 0:
                break
            else:
                cycle_time += 1
        num_sigma.append(cycle_time)
    return num_sigma

# this function estimate the numerator, i.e. the indicator expectation under change of measure with likelihood 
# return the list of the estimate
def num_estimate(lam, mu, gamma, theta, n) -> list:

    num_waiting = []
    
    # change of measure
    lam_new = lam + theta
    mu_new = mu - theta
    
    # generate cycles
    for i in range(n):
        exceed_indicator = 0
        W = 0
        likelihood = 1
        threshold = True
        while True:
            if threshold:
                V = np.random.exponential(scale=1/mu_new, size=1)
                T = np.random.exponential(scale=1/lam_new, size=1)
                W = max(0, W+V-T)
            else:
                V = np.random.exponential(scale=1/mu, size=1)
                T = np.random.exponential(scale=1/lam, size=1)
                W = max(0, W+V-T)
            if W == 0: break
            else:
                if threshold:
                    likelihood = likelihood * (lam*np.exp(-lam*T)*mu*np.exp(-mu*V))/(lam_new*np.exp(-lam_new*T)*mu_new*np.exp(-mu_new*V))
                if W >= gamma:
                    threshold = False
                    exceed_indicator += likelihood[0]
        num_waiting.append(exceed_indicator)
    return num_waiting

# estimate the tail probability
# return the numerator estimate, denominator estimate, and the ratio
def importance_sampling_with_change(lam, mu, gamma, theta, n) -> dict:
    waiting = num_estimate(lam, mu, gamma, theta, n)
    sigma = deno_estimate(lam, mu, n)
    num_mean = np.mean(waiting)
    num_std = np.std(waiting)
    den_mean = np.mean(sigma)
    den_std = np.mean(sigma)
    prob_mean = num_mean/den_mean
    S = np.sqrt(num_std**2-2*prob_mean*np.cov(waiting, sigma)[0][1]+den_std**2*(prob_mean**2))

    rela_error_num = num_std/np.sqrt(n)/num_mean
    theo_prob = lam/mu*np.exp(-(mu-lam)*gamma)
    rela_error_prob = S/den_mean/np.sqrt(n)/prob_mean

    return {"num_mean": num_mean, "95% num_CI": [num_mean-1.96*num_std/np.sqrt(n), num_mean+1.96*num_std/np.sqrt(n)],"num_RE": rela_error_num,"den_mean": den_mean, "prob_mean": prob_mean, "95% prob_CI": [prob_mean-1.96*S/den_mean/np.sqrt(n), prob_mean+1.96*S/den_mean/np.sqrt(n)], "Prob_RE": rela_error_prob}


Now we try another two change of measure about $\theta$ near the optimal solution, that is, $\frac{\lambda}{\lambda + \theta}\times \frac{\mu}{\mu-\theta} = 0.9$ or $1.1$ in the light traffic case, and  $\frac{\lambda}{\lambda + \theta}\times \frac{\mu}{\mu-\theta} = 0.99$ or $1.1$ for the heavy traffic case.

In [43]:
from sympy import *
def solve_theta(lam, mu, value):
    x = symbols('x', positive=True) # restrict the solution to be positive
    return solve(lam/(lam+x)*mu/(mu-x)-value, x)
solve_theta(0.9, 2.7, 1)
solve_theta(0.9, 2.7, 0.9)
solve_theta(0.9, 2.7, 1.1)
solve_theta(1.6, 2, 1)
solve_theta(1.6, 2, 0.99)
solve_theta(1.6, 2, 1.01)

[1.80000000000000]

[0.165153077165047, 1.63484692283495]

[1.91533693467198]

[0.400000000000000]

[0.112382834576964, 0.287617165423036]

[0.467737125398835]

The above result shows that in the light traffic case, we can try $\theta = 1.6348$ or $1.91534$ and in the heavy traffic case, we can try $\theta = 0.77524$ or $0.287617$. Now we try the threshold $\gamma$ such that the tail probability would be $0.001, 10^{-5}, 10^{-10}$ under the optimal change of measure. In the light traffic case, we obtain that when $\gamma = 3.2273,5.78573,12.181799$, the tail probability would be $0.001, 10^{-5}, 10^{-10}$ under different changes of measure. This is in accordance with the theoretical solution.

In [44]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 3.2273, 1.6348, 100000)

{'num_mean': 0.0014988018117590028,
 '95% num_CI': [0.0014815411154416496, 0.001516062508076356],
 'num_RE': 0.005875678587951486,
 'den_mean': 1.49548,
 'prob_mean': 0.0010022212344926063,
 '95% prob_CI': [0.0009891083257715617, 0.001015334143213651],
 'Prob_RE': 0.006675431852070469}

In [45]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 3.2273, 1.8, 100000)

{'num_mean': 0.0014946506056786203,
 '95% num_CI': [0.0014785413481964759, 0.0015107598631607646],
 'num_RE': 0.005498949980841687,
 'den_mean': 1.49514,
 'prob_mean': 0.0009996726765912358,
 '95% prob_CI': [0.000987219987172545, 0.0010121253660099266],
 'Prob_RE': 0.006355493270441424}

In [46]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 3.2273, 1.91534, 100000)

{'num_mean': 0.001495736240648181,
 '95% num_CI': [0.0014784399571564355, 0.0015130325241399264],
 'num_RE': 0.005899860011909449,
 'den_mean': 1.49655,
 'prob_mean': 0.0009994562431246406,
 '95% prob_CI': [0.0009863284634557946, 0.0010125840227934865],
 'Prob_RE': 0.006701490751490671}

In [47]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 5.78573, 1.6348, 100000)

{'num_mean': 1.5076809772005212e-05,
 '95% num_CI': [1.4893287560291367e-05, 1.5260331983719055e-05],
 'num_RE': 0.006210450546409058,
 'den_mean': 1.50148,
 'prob_mean': 1.0041299099558578e-05,
 '95% prob_CI': [9.903930147203908e-06, 1.0178668051913247e-05],
 'Prob_RE': 0.006979794096964511}

In [48]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 5.78573, 1.8, 100000)

{'num_mean': 1.5066076099803132e-05,
 '95% num_CI': [1.4905408869572123e-05, 1.522674333003414e-05],
 'num_RE': 0.005440904194659195,
 'den_mean': 1.50559,
 'prob_mean': 1.0006758878448404e-05,
 '95% prob_CI': [9.883071433457934e-06, 1.0130446323438873e-05],
 'Prob_RE': 0.0063063215619958}

In [49]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 5.78573, 1.91534, 100000)

{'num_mean': 1.5015246008953483e-05,
 '95% num_CI': [1.4835352373013885e-05, 1.5195139644893081e-05],
 'num_RE': 0.006112618285534039,
 'den_mean': 1.50509,
 'prob_mean': 9.976311057115177e-06,
 '95% prob_CI': [9.841805907542706e-06, 1.0110816206687649e-05],
 'Prob_RE': 0.006878802787884401}

In [50]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 12.181799, 1.6348, 100000)

{'num_mean': 1.4980136772219786e-10,
 '95% num_CI': [1.4767094753840843e-10, 1.519317879059873e-10],
 'num_RE': 0.007255935575819755,
 'den_mean': 1.50457,
 'prob_mean': 9.956423943199576e-11,
 '95% prob_CI': [9.802004919215797e-11, 1.0110842967183355e-10],
 'Prob_RE': 0.007913003380301686}

In [51]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 12.181799, 1.8, 100000)

{'num_mean': 1.4821210336073484e-10,
 '95% num_CI': [1.466293690655783e-10, 1.4979483765589139e-10],
 'num_RE': 0.00544839105051648,
 'den_mean': 1.50465,
 'prob_mean': 9.850271050459232e-11,
 '95% prob_CI': [9.728760774961986e-11, 9.971781325956478e-11],
 'Prob_RE': 0.006293739350057068}

In [52]:
np.random.seed(43)
importance_sampling_with_change(0.9, 2.7, 12.181799, 1.91534, 100000)

{'num_mean': 1.504899900934638e-10,
 '95% num_CI': [1.4842549495800974e-10, 1.5255448522891787e-10],
 'num_RE': 0.006999228612914704,
 'den_mean': 1.50041,
 'prob_mean': 1.002992449353602e-10,
 '95% prob_CI': [9.878867806031636e-11, 1.0180981181040405e-10],
 'Prob_RE': 0.0076839799314854995}

In the heavy traffic case, we obtain that when $\gamma = 16.711529,28.22445478,57.00676845$, the tail probability would be $0.001, 10^{-5}, 10^{-10}$ under different changes of measure.

In [53]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 16.711529, 0.287617, 100000)

{'num_mean': 0.0049337170804164944,
 '95% num_CI': [0.004735946160711278, 0.005131488000121711],
 'num_RE': 0.020451827459333594,
 'den_mean': 4.98232,
 'prob_mean': 0.0009902449221279434,
 '95% prob_CI': [0.0009501137609576281, 0.0010303760832982586],
 'Prob_RE': 0.020676785886216548}

In [54]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 16.711529, 0.4, 100000)

{'num_mean': 0.005031184679157149,
 '95% num_CI': [0.0049100294439076335, 0.005152339914406665],
 'num_RE': 0.012286151170627018,
 'den_mean': 5.0439,
 'prob_mean': 0.0009974790696003388,
 '95% prob_CI': [0.0009727757712192522, 0.0010221823679814254],
 'Prob_RE': 0.012635577074183271}

In [55]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 16.711529, 0.467737125398835, 100000)

{'num_mean': 0.0050125486435099575,
 '95% num_CI': [0.004892335707014254, 0.005132761580005661],
 'num_RE': 0.012235917340087407,
 'den_mean': 4.99355,
 'prob_mean': 0.001003804636683313,
 '95% prob_CI': [0.0009788595033257963, 0.0010287497700408295],
 'Prob_RE': 0.012678870360598947}

In [56]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 16.711529, 0.77524, 100000)

{'num_mean': 0.003964337551309024,
 '95% num_CI': [0.0032689691051241533, 0.004659705997493894],
 'num_RE': 0.08949283830912143,
 'den_mean': 4.98652,
 'prob_mean': 0.0007950108595391222,
 '95% prob_CI': [0.0006554697608290313, 0.0009345519582492132],
 'Prob_RE': 0.08955152909315685}

In [57]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 28.22445478, 0.287617, 100000)

{'num_mean': 5.002432778245905e-05,
 '95% num_CI': [4.723956702708743e-05, 5.280908853783066e-05],
 'num_RE': 0.028402106869674475,
 'den_mean': 5.02052,
 'prob_mean': 9.963973409618734e-06,
 '95% prob_CI': [9.405672259693007e-06, 1.052227455954446e-05],
 'Prob_RE': 0.028587744443128605}

In [58]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 28.22445478, 0.4, 100000)

{'num_mean': 4.963833340687203e-05,
 '95% num_CI': [4.843370253606917e-05, 5.084296427767489e-05],
 'num_RE': 0.012381712780454232,
 'den_mean': 5.00206,
 'prob_mean': 9.923578167169532e-06,
 '95% prob_CI': [9.674678674796374e-06, 1.017247765954269e-05],
 'Prob_RE': 0.012796748792205244}

In [59]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 28.22445478, 0.77524, 100000)

{'num_mean': 3.508940529337968e-05,
 '95% num_CI': [1.320939850514163e-05, 5.696941208161772e-05],
 'num_RE': 0.31813787313218966,
 'den_mean': 4.99489,
 'prob_mean': 7.025060670681372e-06,
 '95% prob_CI': [2.6442403566525147e-06, 1.1405880984710229e-05],
 'Prob_RE': 0.31816271914132477}

In [60]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 28.22445478, 0.467737125398835, 100000)

{'num_mean': 4.852160613904955e-05,
 '95% num_CI': [4.708688524522257e-05, 4.995632703287653e-05],
 'num_RE': 0.015086072252770512,
 'den_mean': 4.93848,
 'prob_mean': 9.8252106192694e-06,
 '95% prob_CI': [9.52853533481649e-06, 1.012188590372231e-05],
 'Prob_RE': 0.015405770615292781}

In [61]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 57.00676845, 0.287617, 100000)

{'num_mean': 4.6518021459142134e-10,
 '95% num_CI': [4.220047320575072e-10, 5.083556971253355e-10],
 'num_RE': 0.04735435154870096,
 'den_mean': 4.96371,
 'prob_mean': 9.371623535448714e-11,
 '95% prob_CI': [8.49982143497978e-11, 1.024342563591765e-10],
 'Prob_RE': 0.047462106043067165}

In [62]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 57.00676845, 0.4, 100000)

{'num_mean': 5.084921576597931e-10,
 '95% num_CI': [4.962385421818115e-10, 5.207457731377747e-10],
 'num_RE': 0.012294869325803905,
 'den_mean': 4.95758,
 'prob_mean': 1.0256862373573258e-10,
 '95% prob_CI': [1.0001472421459041e-10, 1.0512252325687475e-10],
 'Prob_RE': 0.012703787106704346}

In [63]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 57.00676845, 0.77524, 100000)

{'num_mean': 2.6530871308157046e-10,
 '95% num_CI': [8.650387395167509e-12, 5.219670387679735e-10],
 'num_RE': 0.4935688836949164,
 'den_mean': 4.9961,
 'prob_mean': 5.310316308351924e-11,
 '95% prob_CI': [1.7289029361761949e-12, 1.0447742323086229e-10],
 'Prob_RE': 0.4935931439113643}

In [64]:
np.random.seed(43)
importance_sampling_with_change(1.6, 2, 57.00676845, 0.467737125398835, 100000)

{'num_mean': 5.055186667565153e-10,
 '95% num_CI': [4.740195547773568e-10, 5.370177787356739e-10],
 'num_RE': 0.03179106244025471,
 'den_mean': 5.0054,
 'prob_mean': 1.0099465911945406e-10,
 '95% prob_CI': [9.466966609881008e-11, 1.0731965214009803e-10],
 'Prob_RE': 0.031952553566359766}